In [166]:
import numpy as np
from enum import Enum
from queue import PriorityQueue
from math import sqrt

dont end when find the solution (stop when? when total_cost == minimum_cost_found)
divide h+c (first one to go out in queue) with tot cost (change the path in branch)

In [167]:
# grid = np.array([
#     [0, 1, 0, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0],
#     [0, 1, 0, 1, 0, 0],
#     [0, 0, 0, 1, 1, 0],
#     [0, 0, 0, 1, 0, 0],
# ])
grid = np.array([
    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0],
    [0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
])

start = (1, 0)
goal = (7, 8)

In [168]:
# Heuristic Map
def dist(pos, goal):
  return sqrt(abs(goal[0]-pos[0])**2 + abs(goal[1]-pos[1])**2)

In [169]:
class Action(Enum):
    # (i, j, cost)
    LEFT = (0, -1, 1)
    RIGHT = (0, 1, 1)
    UP = (-1, 0, 1)
    DOWN = (1, 0, 1)
    UR_DIAG = (-1, 1, 1.41)
    UL_DIAG = (-1, -1, 1.41)
    DR_DIAG = (1, 1, 1.41)
    DL_DIAG = (1, -1, 1.41)

    def __str__(self):
        if self == self.LEFT:
            return '<'
        elif self == self.RIGHT:
            return '>'
        elif self == self.UP:
            return '^'
        elif self == self.DOWN:
            return 'v'
        elif self == self.UR_DIAG:
            return '/^'
        elif self == self.UL_DIAG:
            return '\\^'
        elif self == self.DR_DIAG:
            return '\\v'
        elif self == self.DL_DIAG:
            return '/v'
        
    @property
    def cost(self):
        return self.value[2]
    
    @property
    def delta(self):
        return (self.value[0], self.value[1])

In [170]:
def visualize(grid, path, start):
  x, y = start
  grid[x][y] = 8
  for act in path:
    x += act.delta[0]
    y += act.delta[1]
    grid[x][y] = 8

  print(grid)
  

In [171]:
# Reurns the valid moves depending on the current node
def valid_moves(grid, current):
  valid = []
  for act in Action:
    delta = act.value
    next_pos = (current[0] + delta[0], current[1] + delta[1])

    is_inside_grid = (next_pos[0] >= 0 and next_pos[0] < len(grid)) and (next_pos[1] >= 0 and next_pos[1] < len(grid[0])) 
    
    if is_inside_grid and grid[next_pos] != 1:
      valid.append(act)
      
  return valid

In [172]:
# A star
def Astar(grid, start, goal):
    q = PriorityQueue()
    q.put((dist(start, goal), start))

    visited = set()
    visited.add(start)

    branch = {}
    
    i = 0
    while True:
        curr = q.get()
        current_cost = curr[0]
        current_pos = curr[1]
        
        if current_pos == goal:
            break

        # print('A')
        # print(current_pos)
        # print('NEW MOVES:')

        for action in valid_moves(grid, current_pos):
            i+=1
            delta = action.delta

            new_pos = (current_pos[0] + delta[0], current_pos[1] + delta[1])
            new_cost = current_cost + action.cost + dist(new_pos, goal)

            # print(new_pos, new_cost)
            if new_pos not in visited:                
                visited.add(new_pos)                                   # 1. Mark it as visited           
                q.put((new_cost, new_pos))                            # 2. Add it to the queue
                branch[new_pos] = (current_pos, action, new_cost)     # 3. Add how to get there to branch

            # elif new_pos in branch and branch[new_pos][2] > new_cost:  # if cost is less, update it
            #     branch[new_pos] = (current_pos, action, new_cost)

        # print(visited)
        # print(q)
        # print(branch)
            
    
    # Retrace path
    path = []
    n = goal
    # tot_cost = branch[n][2]
    
    while branch[n][0] != start:
        # append each new node to the path
        path.append(branch[n][1])
        n = branch[n][0]

    # append the start location
    path.append(branch[n][1])
    print(i)
    # reverse the path
    return path[::-1]

In [173]:
path = Astar(grid, start, goal)

392


In [174]:
# PRINT RESULT
visualize(grid, path, start)
# print(grid)
# for act in path:
#     print(act, end=' ')
# print()
# print(cost)

[[0 1 0 0 0 0 0 1 0 8 0 0]
 [8 1 0 0 0 0 0 0 8 1 8 0]
 [8 1 0 1 0 0 0 8 0 1 8 0]
 [0 8 0 1 1 0 8 1 0 1 8 0]
 [0 0 8 1 0 8 0 1 0 1 8 0]
 [0 1 0 8 8 0 0 1 0 1 8 0]
 [0 1 0 0 0 0 0 1 1 1 8 0]
 [0 1 0 1 0 0 0 1 8 8 1 0]
 [0 0 0 1 1 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 0 0 0]]
